In [73]:
#import libraries
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import numpy as np
# import required libraries
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.mplot3d import Axes3D

#import data
matches = pd.read_csv('./dota-2-matches/test_player.csv')
champions = pd.read_csv('./dota-2-matches/hero_names.csv')

#EDA
matches.describe()

,match_id,account_id,hero_id,player_slot
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,99999.500000,91028.072414,50.960134,66.000000
std,28867.527892,103566.804928,33.229504,64.015655
min,50000.000000,0.000000,0.000000,0.000000
25%,74999.750000,0.000000,21.000000,2.000000
50%,99999.500000,39462.500000,48.000000,66.000000
75%,124999.250000,177762.000000,75.000000,130.000000
max,149999.000000,330514.000000,112.000000,132.000000


In [4]:
#remove account_id column
matches = matches.drop("account_id",axis=1)

In [5]:
matches.describe()

,match_id,hero_id,player_slot
count,1000000.000000,1000000.000000,1000000.000000
mean,99999.500000,50.960134,66.000000
std,28867.527892,33.229504,64.015655
min,50000.000000,0.000000,0.000000
25%,74999.750000,21.000000,2.000000
50%,99999.500000,48.000000,66.000000
75%,124999.250000,75.000000,130.000000
max,149999.000000,112.000000,132.000000


In [6]:
#understanding player slot
matches.head(n=20)

,match_id,hero_id,player_slot
0,50000,96,0
1,50000,84,1
2,50000,46,2
3,50000,85,3
4,50000,39,4
5,50000,9,128
6,50000,75,129
7,50000,106,130
8,50000,74,131
9,50000,62,132


In [63]:
#collapse data into columns
competitor_rows = pd.pivot_table(matches, values='hero_id', index=['match_id'], columns='player_slot')
competitor_rows.head(n=10)


player_slot,0,1,2,3,4,128,129,130,131,132
match_id,,,,,,,,,,
50000,96,84,46,85,39,9,75,106,74,62
50001,44,10,57,2,106,58,61,21,18,14
50002,74,7,42,99,88,69,8,25,26,79
50003,44,15,110,56,94,2,101,32,7,72
50004,98,26,73,51,46,2,106,50,65,21
50005,16,39,36,71,46,27,106,33,21,93
50006,104,44,21,62,73,86,19,74,76,67
50007,68,23,2,76,39,65,73,21,9,30
50008,112,11,39,104,101,55,19,16,79,63


In [70]:
# df = df.sort_index() 
# competitor_rows.shape[0]
section = competitor_rows
sectionA,sectionB  = np.split(section, 2, axis=1)
sectionB.columns = sectionA.columns
sectionA.append(sectionB,ignore_index = True)

# sectionB

player_slot,0,1,2,3,4
0,96,84,46,85,39
1,44,10,57,2,106
2,74,7,42,99,88
3,44,15,110,56,94
4,98,26,73,51,46
5,16,39,36,71,46
6,104,44,21,62,73
7,68,23,2,76,39
8,112,11,39,104,101
9,8,5,42,3,92


In [71]:
dataset=[]

for row in sectionA.iterrows():
    index, data = row
    dataset.append(data.tolist())
#it transforms the input dataset (a Python list of lists) into a one-hot encoded NumPy boolean array
oht = TransactionEncoder()
oht_ary = oht.fit(dataset).transform(dataset)
df = pd.DataFrame(oht_ary, columns=oht.columns_)
df.head(n=10)

,0,1,2,3,4,5,6,7,8,9,...,102,103,104,105,106,107,109,110,111,112
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
7,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
9,False,False,False,True,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [86]:
df = df*1
df.head()

,0,1,2,3,4,5,6,7,8,9,...,102,103,104,105,106,107,109,110,111,112
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:

# create kmeans object and call fit
KM = KMeans(n_clusters=6, init='k-means++', random_state=170)
KM = KM.fit(dataset)
print("The cluster centroids are: \n", KM.cluster_centers_)
print("Cluster_label:\n", KM.labels_)
print("Sum of distances of samples to their closest cluster center: \n", KM.inertia_)

#research how kmeans library interprets a binary datasets
#look into the above functions (ex. lablels_)
#find more useful functions


The cluster centroids are: 
 [[ 55.7926461   25.65444964  53.05015099  25.07513766  22.21019599]
 [ 44.92269283  87.11572578  21.76749104  39.20613288  48.214827  ]
 [ 87.81463878  46.52160963  46.89987326  84.16140684  48.4230038 ]
 [ 22.00266899  35.51169133  48.76884247  86.36211198  48.69474906]
 [ 51.44492812  29.74339882  50.07680852  29.49911221  88.39429521]
 [ 44.35513001  83.78512873  87.86883566  43.17964647  48.09798898]]
Cluster_label:
 [2 4 2 ..., 4 0 2]
Sum of distances of samples to their closest cluster center: 
 316239837.781


,0,1,2,3,4,5,6,7,8,9,...,102,103,104,105,106,107,109,110,111,112
0,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,False
1,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2,False,False,False,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,True,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
4,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False


In [ ]:
# from sklearn.cluster import KMeans
# model = KMeans(n_clusters = 3)
# #2D array old data
# model.fit(points)
# # new data
# labels = model.predict(points)
# print(labels)
# resultMatrix = pd.DataFrame(columns = dataRows)
# for matchId in len(matchIdArr):
#     row = [0]*116
#     row[0] = matchIdArr[matchId]
#     for index in range(1,115):
# dataRows = [0]*116
# matchIdArr = []
# for row in matches.itertuples():
#     matchIdArr.append(row.match_id)
#     print(type(row))
# for i in range(len(dataRows)): 
#     dataRows[i] = i
# dataRows[0] = 'Match Id'
# colors = ['blue','red']
# fig = plt.figure(1, figsize=(5, 5))
# ax = Axes3D(fig, elev=-150, azim=110)
# ax.scatter(dataset.grade, 
#            dataset.attend_class, 
#            dataset.extra_hours, 
#            c=KM.labels_, 
#            cmap=matplotlib.colors.ListedColormap(colors),
#            s=75)
# plt.show()
# plt.scatter(dataset.grade, 
#             dataset.extra_hours, 
#             c=KM.labels_, 
#             cmap=matplotlib.colors.ListedColormap(colors), 
#             s=75)
# plt.show()